# Transfomaciones de los datos
---

In [1]:
import pandas as pd
import re

### Carga de los Datasets

In [2]:
credits=pd.read_csv('Dataset/credits.csv')
movies=pd.read_csv('Dataset/movies_dataset.csv')

/tmp/ipykernel_7855/528740004.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies=pd.read_csv('Dataset/movies_dataset.csv')


### Funcion para desanidar cadenas

In [3]:
def extrac_data(df, columna):
    def extrac_data(valor):
        if isinstance(valor, str):
            pattern = r"'name': '([^']*)'"
            coincidencias = re.findall(pattern, valor)
            return coincidencias
        elif isinstance(valor, list):
            valores = []
            for elemento in valor:
                valores.extend(extrac_data(elemento))
            return valores
        elif isinstance(valor, dict):
            valores = []
            for clave, elemento in valor.items():
                if clave == 'name':
                    valores.append(elemento)
                elif isinstance(elemento, (str, list, dict)):
                    valores.extend(extrac_data(elemento))
            return valores
        return None

    df[columna] = df[columna].apply(extrac_data)

    return df


### Aplicamos la funcion para desanidar sobre los campos necesarios

In [4]:
# Aplicar la función a las columnas "belongs_to_collection" y "production_companies"
movies = extrac_data(movies, 'belongs_to_collection')
movies = extrac_data(movies, 'production_companies')

### Los valores nulos de los campos **`revenue`**, **`budget`** son ser rellenados por el número 0.

In [5]:
movies['revenue'].fillna(0, inplace=True)
movies['budget'].fillna(0, inplace=True)

### Los valores nulos del campo **`release date`** deben eliminarse

In [6]:
# Patrón de fecha que consiste en cuatro dígitos seguidos de un guion, dos dígitos y otro guion, y dos dígitos más
pattern = r'^\d{4}-\d{2}-\d{2}$'

# Convertir la columna 'release_date' a tipo de datos string (cadena)
movies['release_date'] = movies['release_date'].astype(str)

# Filtrar las filas que no coinciden con el patrón de fecha
non_matching_movies = movies[~movies['release_date'].str.contains(pattern)]

# Eliminar las filas que no coinciden con el patrón del DataFrame 'movies'
movies = movies.drop(non_matching_movies.index)

# Eliminar los valores nulos restantes en la columna 'release_date'
movies['release_date'].dropna(inplace=True)

# Comprobar la cantidad de valores nulos en la columna 'release_date'
movies['release_date'].isnull().sum()


0

###  Creamos la columna **`release_year`** donde extraerán el año de la fecha de estreno.

In [7]:
# Convierte la columna 'release_date' a tipo de dato datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Extrae el año de la columna 'release_date' y asignalo a una nueva columna 'release_year'
movies['release_year'] = movies['release_date'].dt.year

In [8]:
# Convertir la columna 'budget' a tipo float
movies['budget'] = movies['budget'].astype(float)

# Calcular el retorno de inversión y asignar el valor 0 cuando no hay datos disponibles
movies['return'] = movies['revenue'].div(movies['budget'], fill_value=0)

# Remplazar NaN por 0
movies['return']= movies['return'].fillna(0)



###   Cree la columna con el retorno de inversion, dividiendo estas dos columnas: **`revenue / budget`**.

In [9]:
# Verificar y eliminar filas con valores no numéricos o nulos en la columna 'budget'
movies = movies[pd.to_numeric(movies['budget'], errors='coerce').notnull()]
# Convertir la columna 'budget' a tipo de dato float
movies['budget'] = movies['budget'].astype(float)
# Calcular el ratio entre 'revenue' y 'budget' y asignarlo a la columna 'return'
movies['return'] = movies['revenue'] / movies['budget']



### Elimino las columnas que no serán utilizadas, **`video`**,**`imdb_id`**,**`adult`**,**`original_title`**,**`poster_path`** y **`homepage`**.

In [10]:
# Eliminar múltiples columnas
columnas_a_eliminar = ['video', 'imdb_id', 'adult','original_title','poster_path','homepage']
movies = movies.drop(columnas_a_eliminar, axis=1)

<br></br>


### Preparo el dataframe de **`Movies`** para mergearlo con **`Credits`** 

In [11]:
#convierto el id de object a Int
movies['id']=movies['id'].astype(int)

### Se hace el Merge en base al campo Id

In [12]:
movies_Credits_Merged = movies.merge(credits, on="id")

### Creo el Csv final sin ninguna otra transformacion que las que se solicitan

In [13]:
movies_Credits_Merged.to_csv('Dataset/Movies_Credits_Merged.csv',index=False)